# Project 1
**Finding the Higgs Boson**
![Higgs Boson](https://home.cern/sites/home.web.cern.ch/files/styles/medium/public/image/featured/2014/01/higgs_event_display_1.jpg?itok=0zdD0QEr)

## Importing libs and modules

In [99]:
import numpy as np
from proj1_helpers import *
from implementations import *
from plots import *

train_path = './data/train.csv'
test_path = './data/test.csv'

In [100]:
y, x, ids = load_csv_data(train_path)
x, mean_x, std_x = standardize(x)

In [101]:
y, tx = build_model_data(x, y)

In [102]:
train_x, train_y, test_x, test_y = split_data(tx, y, 0.9, seed=1)

In [103]:
degree = 12
ratio = 0.9
px_train = build_poly(degree=degree,x=train_x)
px_test = build_poly(degree=degree, x=test_x)
rmse_tr = []
rmse_te = []
ws = ridge_regression(lambda_=0.001, tx=px_train, y=train_y)

(373, 373)


In [104]:
y_pred = predict_labels(ws, px_test)

In [105]:
accuracy = 1 - np.mean( y_pred != test_y )

In [106]:
print("Accuracy: " + str(accuracy) + "%")

Accuracy: 0.78852%
